In [ ]:
%cd ..
import os
import time

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

In [ ]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
DATA_DIR = "/data/processed/full-dataset/trainval"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
RAW_DATA_DIR = "/data/raw/full-dataset/trainval"

In [ ]:
import cv2
from os.path import exists
def construct_video(frames, name, size=(1600, 900), base=False, vid_name=None, vid_prefix=False):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/" + name + ".mp4"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        if vid_prefix:
            img_path = os.path.join(RAW_DATA_DIR, 'sweeps/CAM_FRONT', frame)
            if not exists(img_path):
                img_path = os.path.join(RAW_DATA_DIR, 'samples/CAM_FRONT', frame)
        else:
            img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [ ]:
########################## QUERY 1 ##########################

In [ ]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
cam = camera

world = world.filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    F.contains_all('intersection', [obj1.trans]@cam.time)
)


In [ ]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT', 'scene-0245-CAM_FRONT'], base=False)

optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

In [ ]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results, name="query-1")

In [ ]:
########################## QUERY 2 ##########################

In [ ]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
cam = camera

world = world.filter(
    (F.like(obj1.type, 'truck')) &
    (F.convert_camera(obj1.traj@cam.time, cam.ego) > [-10, 0]) &
    (F.convert_camera(obj1.traj@cam.time, cam.ego) < [-1, 50])
)


In [ ]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT', 'scene-0245-CAM_FRONT'], base=False)

optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

In [ ]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results, name="query-2")